In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
q_votes = pd.DataFrame.from_csv('question-votes.csv', index_col=None)
q_votes.head(8)

,VoteType,CreationDate,AnsId
0,2,2011-11-15T00:00:00.000,1
1,2,2011-11-15T00:00:00.000,1
2,2,2011-11-16T00:00:00.000,1
3,2,2011-11-16T00:00:00.000,1
4,2,2011-11-16T00:00:00.000,1
5,2,2011-11-22T00:00:00.000,1
6,2,2011-11-28T00:00:00.000,1
7,2,2011-12-13T00:00:00.000,1
